# Exercise 05


In [ ]:
%matplotlib inline
import numpy as np
from scipy import sparse as sp
import lanczos
from matplotlib import pyplot as plt

In [ ]:
L = 14
g = 1.5
H = lanczos.gen_hamiltonian_L(L, g)
# iteration_psi_0 = np.random.choice([0, 1], size=H.shape[0], replace=True)
iteration_psi_0 = np.random.random(H.shape[0])
iteration_psi_0 /= np.linalg.norm(iteration_psi_0)

In [ ]:
# since g != 1, we do NOT expect ground state degeneracy
T, V = lanczos.lanczos(iteration_psi_0, H, N=200, stabilize=True)
T_unstable, V_unstable = lanczos.lanczos(iteration_psi_0, H, N=200, stabilize=False)

In [ ]:
# Check if V is orthonormal
# normal
assert np.allclose([np.linalg.norm(v) for v in V], 1)
# orthogonal
for i, Vi in enumerate(V):
    for j in range(i):
        assert abs(np.inner(Vi, V[j])) < 1e-13

In [ ]:
eigvals = np.linalg.eigvalsh(T)
unstable_eigvals = np.linalg.eigvalsh(T_unstable)
sp_eigvals, _ = sp.linalg.eigsh(T, k=10, which="SA")

In [ ]:
def eigenval_hist(vals: np.ndarray, method: str = ""):
    plt.figure()
    plt.hist(vals, bins=np.unique(vals).size)
    plt.xlabel("Eigenvalues")
    plt.ylabel("Count")
    plt.title(f"Eigenvalue degeneracy {method}")
    plt.show()
eigenval_hist(eigvals[:10], "lanczos")
eigenval_hist(unstable_eigvals[:10], "lanczos (unstable)")
eigenval_hist(sp_eigvals, "scipy")

As can be seen in the above plots, the "stabilize"-option removes the discrepancy between the scipy (arpack) implementation and the homemade lanczos implementation. With the "stabilize"-parameter set to "True", the algorithm ensures the vectors are orthogonal, whereas with the option disabled the vectors are only normalized. Hence, orthonormality is not fulfilled and ground state degeneracy might not be fulfilled either.

In [ ]:
E, u = np.linalg.eigh(T)
u0 = u[:, 0]
psi0 = np.array(V).T @ u0

E0 = np.inner(psi0.conj(), H @ psi0)
assert np.allclose(E0, E[0])

psi0_var = np.inner(psi0.conj(), (H @ H) @ psi0) - np.inner(psi0.conj(), H @ psi0)**2
assert np.allclose(psi0_var, 0)

In [ ]:
Splus_list = [lanczos.singlesite_to_full(lanczos.Splus, i, L) for i in range(L)]

In [ ]:
def get_x0(z: complex, T: np.ndarray) -> complex:
    alpha = T.diagonal(0)
    beta = T.diagonal(1)
    N = T.shape[0]

    def continued_fraction(i: int) -> float:
        if i == N - 1:
            return 1 / (z - alpha[i])
        return 1 / (z - alpha[i] - beta[i]**2 * continued_fraction(i + 1))
    return continued_fraction(0)

In [ ]:
def get_I(O, z: complex | np.ndarray) -> complex | np.ndarray:
    phi0 = O @ psi0
    T, V = lanczos.lanczos(phi0, H, N=200, stabilize=True)

    x0s = get_x0(z, T)
    return -1 / np.pi * x0s.imag

In [ ]:
omegas = np.linspace(-1, 10, 1000)
epsilon = 0.1
zs = omegas + E0 + 1j * epsilon
I_Splus = get_I(Splus_list[0], zs)

In [ ]:
plt.figure()
plt.plot(omegas, I_Splus)
plt.xlabel("omega")
plt.ylabel("I(z)")
plt.title("$O = S_0^+$")
plt.show()

In [ ]:
def get_Skplus(k: float) -> sp.csr_matrix:
    exp = np.exp(1j * k * np.arange(L))
    return L**-0.5 * np.sum(exp * np.array(Splus_list))

In [ ]:
ks = np.linspace(-L/2, L/2, L+1, endpoint=True) * 2*np.pi / L
Is = []
for k in ks:
    Is.append(get_I(get_Skplus(k), zs))

In [ ]:
plt.figure()
lanczos.colorplot(ks, omegas, Is)
plt.xlabel("k")
plt.ylabel("omega")
plt.title("I(Sk, omega)")
plt.show()